# 04: Extracción de Votaciones (Detalle)

**Propósito:** Este *notebook* extrae el historial detallado de todas las votaciones (`roll-call data`) para cada período legislativo.

**Dependencias:**
* `data/01_raw/periodos_master.csv`: La lista maestra de períodos.

**Salidas (Artifacts):**
* `data/01_raw/[periodo]/detalle.csv`: Un archivo CSV por período que contiene el detalle de cada voto (quién, qué, cómo votó) en cada votación.

In [1]:
import pandas as pd
from pathlib import Path
import sys
import logging
from tqdm.notebook import tqdm # Para barras de progreso

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    # Asumimos que 'sanitize_filename' está en un módulo común
    from src.common_utils import sanitize_filename
    # Importamos la función orquestadora principal de nuestro NUEVO módulo
    from src.vote_utils import build_detalle_periodo 
except ImportError as e:
    logging.error(f"ERROR: No se pudieron importar las funciones desde /src. {e}")
    logging.error("Asegúrese de que 'src/common_utils.py' y 'src/vote_utils.py' existan.")
    raise

# Registrar 'tqdm' con pandas
tqdm.pandas()

In [2]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"

# --- Definir el ARCHIVO DE ENTRADA (Dependencia) ---
MASTER_PERIOD_FILE = DATA_DIR_RAW / "periodos_master.csv"

logging.info(f"Ruta Raíz: {ROOT}")
logging.info(f"Directorio de Datos Raw: {DATA_DIR_RAW}")

2025-10-26 11:53:09,545 - INFO - Ruta Raíz: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-
2025-10-26 11:53:09,550 - INFO - Directorio de Datos Raw: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw


## 2. Carga de Dependencias (Períodos)

Cargamos la lista maestra de períodos. La función de extracción (`build_detalle_periodo`) itera internamente usando el nombre del período.

In [3]:
try:
    df_periodos = pd.read_csv(MASTER_PERIOD_FILE)
    logging.info(f"Se cargó la lista maestra de {len(df_periodos)} períodos.")
    display(df_periodos.head())
except FileNotFoundError as e:
    logging.error(f"ERROR FATAL: No se encontró el archivo de dependencia: {MASTER_PERIOD_FILE}")
    logging.error("Por favor, ejecute el notebook '00_Extraction_Periods.ipynb' primero.")
    raise

2025-10-26 11:53:09,559 - INFO - Se cargó la lista maestra de 10 períodos.


,Id,Nombre,FechaInicio,FechaTermino
0,7,1965-1969,1965-03-11,1969-03-11 00:00:00
1,1,1990-1994,1990-03-11,1994-03-10 00:00:00
2,2,1994-1998,1994-03-11,1998-03-10 00:00:00
3,3,1998-2002,1998-03-11,2002-03-10 00:00:00
4,4,2002-2006,2002-03-11,2006-03-10 00:00:00


## 3. Bucle Principal de Extracción de Votos

Iteramos sobre cada período y llamamos a la función orquestadora `build_detalle_periodo` (que vive en `src/vote_utils.py`). Esta es la parte más lenta del *pipeline*.

In [4]:
logging.info(f"Iniciando extracción de votaciones para {len(df_periodos)} períodos...")

for row in tqdm(df_periodos.itertuples(), total=len(df_periodos), desc="Procesando Períodos"):
    
    # La función 'build_detalle_periodo' (del notebook 02)
    # parece usar el 'Nombre' del período (ej. "1990-1994")
    nombre_periodo = row.Nombre 
    
    nombre_carpeta = sanitize_filename(nombre_periodo)
    carpeta_periodo = DATA_DIR_RAW / nombre_carpeta
    
    # --- Definir ruta de Output ---
    ruta_output_votes = carpeta_periodo / "detalle.csv"

    logging.info(f"--- Procesando Período: {nombre_periodo} ---")

    # --- Chequeo: Evitar Re-procesamiento ---
    if ruta_output_votes.exists():
        logging.info(f"Saltando período: El archivo 'detalle.csv' ya existe.")
        continue

    try:
        # 1. Llamar a la lógica compleja (que está en /src)
        # Esta función debe contener los 'Client' de zeep y toda la lógica
        logging.info("Llamando a build_detalle_periodo... (Esto puede tardar horas)")
        df_out = build_detalle_periodo(nombre_periodo)
        
        if df_out is None or df_out.empty:
            logging.warning(f"No se generaron datos para el período {nombre_periodo}. Archivo NO creado.")
            continue

        # 2. Crear carpeta (si la función no lo hizo)
        carpeta_periodo.mkdir(parents=True, exist_ok=True)

        # 3. Guardar
        df_out.to_csv(ruta_output_votes, index=False, encoding="utf-8")
        logging.info(f"Guardado exitosamente: {ruta_output_votes} ({len(df_out)} filas)")

    except Exception as e:
        logging.error(f"ERROR FATAL al procesar período {nombre_periodo}: {e}", exc_info=True)

logging.info("--- Extracción de votaciones finalizada ---")

2025-10-26 11:53:09,581 - INFO - Iniciando extracción de votaciones para 10 períodos...


Procesando Períodos:   0%|          | 0/10 [00:00<?, ?it/s]

2025-10-26 11:53:09,606 - INFO - --- Procesando Período: 1965-1969 ---
2025-10-26 11:53:09,607 - INFO - Llamando a build_detalle_periodo... (Esto puede tardar horas)


  0%|          | 0/4 [00:00<?, ?it/s]

2025-10-26 11:53:10,043 - INFO - Procesando año: 1965
2025-10-26 11:53:10,080 - INFO - Procesando año: 1966
2025-10-26 11:53:10,120 - INFO - Procesando año: 1967
2025-10-26 11:53:10,152 - INFO - Procesando año: 1968
2025-10-26 11:53:10,190 - ERROR - ERROR FATAL al procesar período 1965-1969: No objects to concatenate
Traceback (most recent call last):
  File "C:\Users\angel\AppData\Local\Temp\ipykernel_9524\110925791.py", line 26, in <module>
    df_out = build_detalle_periodo(nombre_periodo)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\notebooks\..\src\vote_utils.py", line 52, in build_detalle_periodo
    detalles_periodo_df = pd.concat(detalles_list, ignore_index=True)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\angel\anaconda3\Lib\site-packages\pandas\core\reshape\concat.py", line 380, in concat
    op = _Concatenator(
        

...

2025-10-26 12:11:15,687 - WARNING - Error en votacion 36733: 'Votos.Voto'
2025-10-26 12:12:25,136 - INFO - Procesando año: 2024


  0%|          | 0/1844 [00:00<?, ?it/s]

2025-10-26 12:13:58,028 - INFO - Procesando año: 2025


  0%|          | 0/1082 [00:00<?, ?it/s]

2025-10-26 12:15:14,910 - INFO - Guardado exitosamente: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\2022-2026\detalle.csv (783644 filas)
2025-10-26 12:15:14,912 - INFO - --- Extracción de votaciones finalizada ---
